# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 20 2022 12:40PM,249084,10,0086092430,ISDIN Corporation,Released
1,Oct 20 2022 12:40PM,249084,10,0086092428,ISDIN Corporation,Released
2,Oct 20 2022 12:40PM,249084,10,0086092429,ISDIN Corporation,Released
3,Oct 20 2022 12:40PM,249084,10,0086092440,ISDIN Corporation,Released
4,Oct 20 2022 12:40PM,249084,10,0086092439,ISDIN Corporation,Released
5,Oct 20 2022 12:40PM,249084,10,0086092441,ISDIN Corporation,Released
6,Oct 20 2022 12:40PM,249084,10,0086092442,ISDIN Corporation,Released
7,Oct 20 2022 12:40PM,249084,10,0086092451,ISDIN Corporation,Released
8,Oct 20 2022 12:40PM,249084,10,0086092452,ISDIN Corporation,Released
9,Oct 20 2022 12:40PM,249084,10,0086092454,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,249078,Released,1
31,249079,Released,1
32,249080,Released,10
33,249082,Released,14
34,249084,Released,22


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249078,NaN,NaN,1.0
249079,NaN,NaN,1.0
249080,NaN,NaN,10.0
249082,NaN,NaN,14.0
249084,NaN,NaN,22.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248348,0.0,1.0,0.0
248896,0.0,0.0,1.0
248979,0.0,0.0,2.0
249001,2.0,4.0,2.0
249002,0.0,1.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248348,0,1,0
248896,0,0,1
248979,0,0,2
249001,2,4,2
249002,0,1,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248348,0,1,0
1,248896,0,0,1
2,248979,0,0,2
3,249001,2,4,2
4,249002,0,1,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248348,,1,
1,248896,,,1
2,248979,,,2
3,249001,2,4,2
4,249002,,1,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 20 2022 12:40PM,249084,10,ISDIN Corporation
22,Oct 20 2022 12:28PM,249082,10,ISDIN Corporation
36,Oct 20 2022 12:25PM,249080,10,"Methapharm, Inc."
46,Oct 20 2022 12:08PM,248348,21,"NBTY Global, Inc."
47,Oct 20 2022 12:06PM,249079,22,"NBTY Global, Inc."
48,Oct 20 2022 12:06PM,249078,22,"NBTY Global, Inc."
49,Oct 20 2022 12:06PM,249077,22,"NBTY Global, Inc."
50,Oct 20 2022 12:05PM,249076,22,"NBTY Global, Inc."
51,Oct 20 2022 12:02PM,249075,10,Emerginnova
57,Oct 20 2022 11:35AM,249073,16,"SHL Pharma, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 20 2022 12:40PM,249084,10,ISDIN Corporation,,,22
1,Oct 20 2022 12:28PM,249082,10,ISDIN Corporation,,,14
2,Oct 20 2022 12:25PM,249080,10,"Methapharm, Inc.",,,10
3,Oct 20 2022 12:08PM,248348,21,"NBTY Global, Inc.",,1,
4,Oct 20 2022 12:06PM,249079,22,"NBTY Global, Inc.",,,1
5,Oct 20 2022 12:06PM,249078,22,"NBTY Global, Inc.",,,1
6,Oct 20 2022 12:06PM,249077,22,"NBTY Global, Inc.",,,1
7,Oct 20 2022 12:05PM,249076,22,"NBTY Global, Inc.",,,1
8,Oct 20 2022 12:02PM,249075,10,Emerginnova,,,6
9,Oct 20 2022 11:35AM,249073,16,"SHL Pharma, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 20 2022 12:40PM,249084,10,ISDIN Corporation,22,,
1,Oct 20 2022 12:28PM,249082,10,ISDIN Corporation,14,,
2,Oct 20 2022 12:25PM,249080,10,"Methapharm, Inc.",10,,
3,Oct 20 2022 12:08PM,248348,21,"NBTY Global, Inc.",,1,
4,Oct 20 2022 12:06PM,249079,22,"NBTY Global, Inc.",1,,
5,Oct 20 2022 12:06PM,249078,22,"NBTY Global, Inc.",1,,
6,Oct 20 2022 12:06PM,249077,22,"NBTY Global, Inc.",1,,
7,Oct 20 2022 12:05PM,249076,22,"NBTY Global, Inc.",1,,
8,Oct 20 2022 12:02PM,249075,10,Emerginnova,6,,
9,Oct 20 2022 11:35AM,249073,16,"SHL Pharma, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 20 2022 12:40PM,249084,10,ISDIN Corporation,22,,
1,Oct 20 2022 12:28PM,249082,10,ISDIN Corporation,14,,
2,Oct 20 2022 12:25PM,249080,10,"Methapharm, Inc.",10,,
3,Oct 20 2022 12:08PM,248348,21,"NBTY Global, Inc.",,1,
4,Oct 20 2022 12:06PM,249079,22,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 20 2022 12:40PM,249084,10,ISDIN Corporation,22.0,NaN,NaN
1,Oct 20 2022 12:28PM,249082,10,ISDIN Corporation,14.0,NaN,NaN
2,Oct 20 2022 12:25PM,249080,10,"Methapharm, Inc.",10.0,NaN,NaN
3,Oct 20 2022 12:08PM,248348,21,"NBTY Global, Inc.",NaN,1.0,NaN
4,Oct 20 2022 12:06PM,249079,22,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 20 2022 12:40PM,249084,10,ISDIN Corporation,22.0,0.0,0.0
1,Oct 20 2022 12:28PM,249082,10,ISDIN Corporation,14.0,0.0,0.0
2,Oct 20 2022 12:25PM,249080,10,"Methapharm, Inc.",10.0,0.0,0.0
3,Oct 20 2022 12:08PM,248348,21,"NBTY Global, Inc.",0.0,1.0,0.0
4,Oct 20 2022 12:06PM,249079,22,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2739524,68.0,38.0,7.0
15,249070,26.0,16.0,0.0
16,249073,1.0,0.0,0.0
19,996074,17.0,40.0,34.0
21,497244,1.0,1.0,0.0
22,996310,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2739524,68.0,38.0,7.0
1,15,249070,26.0,16.0,0.0
2,16,249073,1.0,0.0,0.0
3,19,996074,17.0,40.0,34.0
4,21,497244,1.0,1.0,0.0
5,22,996310,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,68.0,38.0,7.0
1,15,26.0,16.0,0.0
2,16,1.0,0.0,0.0
3,19,17.0,40.0,34.0
4,21,1.0,1.0,0.0
5,22,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,68.0
1,15,Released,26.0
2,16,Released,1.0
3,19,Released,17.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21,22
Status,,,,,,
Completed,7.0,0.0,0.0,34.0,0.0,0.0
Executing,38.0,16.0,0.0,40.0,1.0,0.0
Released,68.0,26.0,1.0,17.0,1.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21,22
0,Completed,7.0,0.0,0.0,34.0,0.0,0.0
1,Executing,38.0,16.0,0.0,40.0,1.0,0.0
2,Released,68.0,26.0,1.0,17.0,1.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21,22
0,Completed,7.0,0.0,0.0,34.0,0.0,0.0
1,Executing,38.0,16.0,0.0,40.0,1.0,0.0
2,Released,68.0,26.0,1.0,17.0,1.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()